In [ ]:
## TensorFlow Version

df_subset = df_match[['hero_id', 'gold_t']].copy()

df_subset = df_subset.explode('gold_t').reset_index(drop=True)

#df_pivoted = df_subset.pivot_table(index=df_subset.index, columns='hero_id', values='gold_t')

df['gold_t'].apply(lambda x: x.split(','))
#df_subset = df_subset.T
df_subset
#df_pivoted



Splitting a sequence in the first step for prepping data for an LSTM model.
This is following along with : https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/


Below we create a function that will create sample sequences based on the number of steps we feed the model before expecting an output. For example, below uses 3 steps fed to predict 1 step out.

In [ ]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = gold_t
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

The LSTM model expects data in the form of [samples,timesteps,features]. Our current X shape is in the form of [samples, timesteps]. i.e. the first row in X is the first sample being [0 . 170, 260] and its timesteps are 3, one for each column.

 Recall that we are using tensors here (3 Dimensional arrays) and will use .Reshape() methods to conform shape to LSTM.


In this case we have one time series. This equates to 1 "feature" to the LSTM.

For this one feature, we've chopped it into 25 samples of 3 time-steps (what we chose in the split_sequence function).

Thus we re-shape our X from [25, 3] into a [25, 3, 1].


In [ ]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

X

In [ ]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32,input_shape=(3, 1))) 

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps,n_features))) # 3 steps, 1 feature
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')